In [ ]:
import sys
import pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import pickle
import itertools
import networkx as nx

In [ ]:
from numba import jit

In [ ]:
import importlib
import evotsc
import evotsc_lib
import evotsc_plot
importlib.reload(evotsc)
importlib.reload(evotsc_lib)
importlib.reload(evotsc_plot)

In [ ]:
label_fontsize=20
tick_fontsize=15
legend_fontsize=15
dpi=300

In [ ]:
exp_path = pathlib.Path('/Users/theotime/Desktop/evotsc/pci/main/')
gen = 1_000_000
gene_types = ['AB', 'A', 'B'] # Name of each gene type
gene_type_color = ['tab:blue', 'tab:red', 'tab:green'] #AB, A, B
orient_name = ['leading', 'lagging'] # Name of each gene orientation
rel_orients = ['conv', 'div', 'downstr', 'upstr'] # In alphabetical order
envs = ['A', 'B'] # Environment names

In [ ]:
rng = np.random.default_rng(seed=123456)

In [ ]:
rep_dirs = sorted([d for d in exp_path.iterdir() if (d.is_dir() and d.name.startswith("rep"))])
nb_reps = len(rep_dirs)
figure_rep = 21
params = evotsc_lib.read_params(rep_dirs[0])
params['m'] = 2.5 # Temporary fix because the parameter wasn't saved

In [ ]:
params

In [ ]:
genes_per_type = params["nb_genes"] / len(gene_types)

# Make random individuals (reused throughout)

In [ ]:
def make_random_indivs(nb_indiv):
    
    mutation = evotsc.Mutation(inversion_poisson_lam=params['inversion_poisson_lam'])
    
    rand_indivs = []
    
    for rep in range(nb_indiv):
        indiv = evotsc_lib.make_random_indiv(intergene=int(params['intergene']),
                                             gene_length=int(params['gene_length']),
                                             nb_genes=int(params['nb_genes']),
                                             default_basal_expression=params['default_basal_expression'],
                                             interaction_dist=params['interaction_dist'],
                                             interaction_coef=params['interaction_coef'],
                                             sigma_basal=params['sigma_basal'],
                                             sigma_opt=params['sigma_opt'],
                                             epsilon=params['epsilon'],
                                             m=params['m'],
                                             selection_coef=params['selection_coef'],
                                             mutation=mutation,
                                             rng=rng,
                                             nb_mutations=100)
        
        indiv.inter_matrix = indiv.compute_inter_matrix()

        rand_indivs.append(indiv)
        
    return rand_indivs

In [ ]:
rand_indivs = make_random_indivs(nb_indiv=nb_reps)

# Evolutionary stats

In [ ]:
def get_stats(exp_name):
    
    exp_name = pathlib.Path(exp_name)
        
    rep_dirs = sorted([d for d in exp_name.iterdir() if (d.is_dir() and d.name.startswith("rep"))])
    
    res = pd.DataFrame()
    
    for i_rep, rep_dir in enumerate(rep_dirs):
        
        res_dir = pd.read_csv(rep_dir.joinpath('stats.csv'),
                              usecols=['Gen', 'Fitness',
                                       'ABon_A', 'ABon_B', 'Aon_A', 'Aon_B', 'Bon_A', 'Bon_B'])

        res_dir.insert(0, 'Replicate', i_rep)
        
        res = pd.concat([res, res_dir])
    
    return res

In [ ]:
full_stats = get_stats(exp_path)

## Plot fitness over evolutionary time

In [ ]:
def plot_fitness(full_stats):
    
    stats = full_stats[full_stats["Gen"] > 0][['Gen', 'Fitness']].copy()
    
    stats['log fitness'] = np.log(stats['Fitness'])
    mean_data = stats.groupby('Gen').mean().reset_index()
    
    # Note: for the fitness, the mean can be above the quantile
    first_dec = stats.groupby('Gen').quantile(0.1)
    last_dec = stats.groupby('Gen').quantile(0.9)
    
    color = mpl.colormaps['viridis'].colors[0]
    
    plt.figure(figsize=(9,4), dpi=dpi)
    
    plt.xscale('log')
    plt.yscale('log')
    plt.grid(linestyle=':')
    plt.grid(visible=True, which="minor", axis='x', linestyle=':')
    
    plt.xlabel('Generation', fontsize=label_fontsize)
    plt.ylabel('Fitness', fontsize=label_fontsize)
    
    plt.tick_params(axis='both', which='major', labelsize=tick_fontsize)
    
    # Average fitness
    plt.plot(mean_data['Gen'],
             np.exp(mean_data['log fitness']),
             color=color,
             linewidth=2) 
    
    # Quantiles
    plt.plot(mean_data['Gen'],
             first_dec['Fitness'],
             color=color,
             alpha=0.3)
    
    plt.plot(mean_data['Gen'],
             last_dec['Fitness'],
             color=color,
             alpha=0.3)

    #plt.title(exp_path.name)
        
    plt.savefig(f'{exp_path}/all_fitness.pdf', dpi=dpi, bbox_inches='tight')

In [ ]:
plot_fitness(full_stats)

# Plot the number of active genes of each type over evolutionary time

In [ ]:
def plot_gene_activity_all(exp_path, full_stats, var_type='quantile'):
    
    mean_data = full_stats.groupby('Gen').mean().reset_index()
    if var_type == 'sigma':
        std_data = full_stats.groupby('Gen').std()
    elif var_type == 'quantile':
        first_dec = full_stats.groupby('Gen').quantile(0.1)
        last_dec = full_stats.groupby('Gen').quantile(0.9)
    elif var_type == 'minmax':
        min_data = full_stats.groupby('Gen').min()
        max_data = full_stats.groupby('Gen').max()
    
    for env in ["A", "B"]:

        fig, ax1 = plt.subplots(figsize=(9, 4), dpi=dpi)
        delta_y = params["nb_genes"] / 3 * 0.05 
        ax1.set_ylim(-delta_y, params["nb_genes"] / 3 + delta_y)
        ax1.set_ylabel('Activated genes', fontsize=label_fontsize)
        ax1.set_xlabel('Generation', fontsize=label_fontsize)
        ax1.set_xscale('log')
        ax1.grid(linestyle=':')
        ax1.grid(visible=True, which="minor", axis='x', linestyle=':')

        for i_gene_type, gene_type in enumerate(gene_types):

            ax1.plot(mean_data['Gen'], mean_data[f"{gene_type}on_{env}"],
                     color=gene_type_color[i_gene_type],
                     linewidth=2,
                     label=gene_type)
            
            # Show 2-sigma (95%) confidence intervals
            if var_type == 'sigma':
                ax1.plot(mean_data['Gen'],
                         mean_data[f"{gene_type}on_{env}"] - 2 * std_data[f"{gene_type}on_{env}"],
                         color=gene_type_color[i_gene_type],
                         alpha=0.3)
                ax1.plot(mean_data['Gen'],
                         mean_data[f"{gene_type}on_{env}"] + 2 * std_data[f"{gene_type}on_{env}"],
                         color=gene_type_color[i_gene_type],
                         alpha=0.3)
            # Show first and last deciles
            elif var_type == 'quantile':
                ax1.plot(mean_data['Gen'],
                         first_dec[f"{gene_type}on_{env}"],
                         color=gene_type_color[i_gene_type],
                         alpha=0.3)
                ax1.plot(mean_data['Gen'],
                         last_dec[f"{gene_type}on_{env}"],
                         color=gene_type_color[i_gene_type],
                         alpha=0.3)
            # Show min and max values
            elif var_type == 'minmax':
                ax1.plot(mean_data['Gen'],
                         min_data[f"{gene_type}on_{env}"],
                         color=gene_type_color[i_gene_type],
                         alpha=0.3)
                ax1.plot(mean_data['Gen'],
                         max_data[f"{gene_type}on_{env}"],
                         color=gene_type_color[i_gene_type],
                         alpha=0.3)


        
        ax1.tick_params(axis='both', which='major', labelsize=tick_fontsize)



        #plt.title(f"Environment {env}")
        fig.legend(bbox_to_anchor=(0, 1),
                   bbox_transform=ax1.transAxes,
                   loc="upper left",
                   fontsize=legend_fontsize)   

        plt.savefig(f'{exp_path}/gene_activity_env_{env}.pdf', dpi=dpi, bbox_inches='tight')

        plt.show()

In [ ]:
plot_gene_activity_all(exp_path, full_stats)

# Plot genomes

In [ ]:
def plot_best_genome_and_tsc(exp_path, gen):
    rep_dirs = sorted([d for d in exp_path.iterdir() if (d.is_dir() and d.name.startswith("rep"))])
    
    for rep_dir in rep_dirs[figure_rep:figure_rep+1]:
        
        rep_num = rep_dir.name[3:]
        
        try:
            best_indiv = evotsc_lib.get_best_indiv(rep_dir, gen)
                        
            evotsc_plot.plot_genome_and_tsc(best_indiv, params['sigma_A'], show_bar=True,
                                            coloring_type='on-off',
                                            id_interval=best_indiv.nb_genes / 12, print_ids=True,
                                        plot_name=exp_path.joinpath(f'genome_and_tsc_rep{rep_num}_env_A.png'))
            
            evotsc_plot.plot_genome_and_tsc(best_indiv, params['sigma_B'], show_bar=False,
                                            coloring_type='on-off',
                                            id_interval=best_indiv.nb_genes / 12, print_ids=True,
                                        plot_name=exp_path.joinpath(f'genome_and_tsc_rep{rep_num}_env_B.png'))
        except FileNotFoundError: # Skip missing data
            pass

In [ ]:
plot_best_genome_and_tsc(exp_path, gen)

# Influence of enviromental supercoiling on final gene expression levels

In [ ]:
nb_sigmas = 250
sigma_min = -0.061
sigma_max = 0.061

In [ ]:
def compute_activity_sigma_per_type(indiv, sigmas):
    
    # Initialize the individual
    indiv.evaluate(0.0, 0.0)

    activ = np.zeros((3, len(sigmas))) # Compute activity for each gene type

    for i_sigma, sigma_env in enumerate(sigmas):
        # Evaluate the individual in the environment with sigma
        temporal_expr = indiv.run_system(sigma_env)

        # Compute total gene activation levels        
        for i_gene, gene in enumerate(indiv.genes):
            activ[gene.gene_type][i_sigma] += temporal_expr[-1, i_gene]
            
    activ /= (indiv.nb_genes / 3)
    
    return activ

In [ ]:
def compute_avg_best_activ_by_sigma():
    rep_dirs = sorted([d for d in exp_path.iterdir() if (d.is_dir() and d.name.startswith("rep"))])
    
    sigmas = np.linspace(sigma_min, sigma_max, nb_sigmas)
    activ = np.zeros((3, len(sigmas)))

    for i_rep in range(nb_reps):
        indiv = evotsc_lib.get_best_indiv(exp_path.joinpath(f'rep{i_rep:02}'), gen=gen)
            
        activ += compute_activity_sigma_per_type(indiv, sigmas)
            
    return activ / nb_reps

In [ ]:
activ = compute_avg_best_activ_by_sigma()

In [ ]:
def generate_rand_activ_by_sigma(rand_indivs):

    sigmas = np.linspace(sigma_min, sigma_max, nb_sigmas)
    activ = np.zeros(len(sigmas))  # Average over all genes, not gene types
    
    for i_rand, rand_indiv in enumerate(rand_indivs):
        activ += np.mean(compute_activity_sigma_per_type(rand_indiv, sigmas), axis=0)
        
    return activ / nb_reps

In [ ]:
rand_activ = generate_rand_activ_by_sigma(rand_indivs)

In [ ]:
# See how gene activity levels depend on environmental supercoiling
def plot_activity_sigma_per_type(activ, rand_activ, plot_title=None, plot_name=None):
    
    sigma_basal = params['sigma_basal']
    sigma_opt = params['sigma_opt']      
    
    sigmas_env = np.linspace(sigma_min, sigma_max, nb_sigmas)

    colors = ['tab:blue', 'tab:red', 'tab:green'] # AB: blue, A: red, B: green
    
    fig, ax = plt.subplots(figsize=(7, 4), dpi=dpi)
    
    plt.xlabel('Background supercoiling ($\sigma_{basal} + \delta\sigma_{env}$)')
    plt.ylabel('Average gene expression by type')
    plt.ylim(-0.05, 1.10)
    plt.xlim(sigmas_env[0] + sigma_basal, sigmas_env[-1] + sigma_basal)
    plt.grid(linestyle=':')
    
    # Add 1/2 expression level
    half_expr = (1 + np.exp(- params['m'])) / 2
    plt.hlines(half_expr, sigmas_env[0] + sigma_basal, sigmas_env[-1] + sigma_basal,
               linestyle=':', linewidth=1.5, color='tab:pink')#, label='Activation threshold')
    
    
    # Add average expression per gene type
    for i_gene_type, gene_type in enumerate(gene_types):
        plt.plot(sigmas_env + sigma_basal, activ[i_gene_type, :],
                 color=gene_type_color[i_gene_type],
                 linewidth=2,
                 label=gene_type)

    # Add sigma_A and sigma_B
    y_min, y_max = plt.ylim()
    plt.vlines(params['sigma_A'] + sigma_basal, y_min, y_max, linestyle='--', linewidth=1, color='black')
    plt.vlines(params['sigma_B'] + sigma_basal, y_min, y_max, linestyle='--', linewidth=1, color='black')
    
    plt.text(params['sigma_A'] + sigma_basal, y_max + 0.005, '$\delta\sigma_A$',
             va='bottom', ha='center', fontsize='large') # Use \mathbf{} for bold
    plt.text(params['sigma_B'] + sigma_basal, y_max + 0.005, '$\delta\sigma_B$',
             va='bottom', ha='center', fontsize='large')
    plt.ylim(y_min, y_max)
        
    # Add expression for an isolated gene
    sigmas_total = sigmas_env + sigma_basal
    activities = 1.0 / (1.0 + np.exp((sigmas_total - sigma_opt)/ params['epsilon']))
    plt.plot(sigmas_env + sigma_basal, np.exp(params['m'] * (activities - 1)),
             linewidth=2, color='tab:cyan', zorder=0, linestyle='--', label='Isolated gene')
    
    # Add expression for a random genome
    plt.plot(sigmas_env + sigma_basal, rand_activ,
         linewidth=2, color='tab:cyan', zorder=0, linestyle=(0, (3, 1, 1, 1)), label='Random')

    plt.legend(loc='lower left')
    
    # Add other ax with other sc
    ax2 = ax.twiny()
    xmin, xmax = ax.get_xlim()
    ax2.set_xlim(xmin - params['sigma_basal'], xmax - params['sigma_basal'])
    ax2.set_xlabel('Environmental shift in supercoiling ($\delta\sigma_{env}$)')
    
    # Wrap up            
    if plot_name:
        plt.savefig(plot_name, dpi=dpi, bbox_inches='tight')
        
    plt.show()
    plt.close()

In [ ]:
plot_activity_sigma_per_type(activ, rand_activ,
                             plot_name=exp_path.joinpath(f'activity_sigmas_avg.pdf'))

# Genome stats: pairs of neighboring genes

In [ ]:
def get_pair_stats(indiv):
        
    result_dict = {'gene_id': [],
                   'gene_type': [],
                   'gene_orient': [],
                   'other_id': [],
                   'other_type': [],
                   'other_orient': [],
                   'rel_orient': [],
                   'intergene_dist': [],
                   'interaction_dist': []}
        
    for i_gene, gene in enumerate(indiv.genes):
        #                                                       other  -  gene
        other = indiv.genes[i_gene - 1]        
        # As in evotsc.compute_inter_matrix(), the relevant distance is
        # the distance between the promoter of the focal gene and the 
        # middle of the other gene.
        if other.orientation == 0 and gene.orientation == 1:   # --|--> <----|
            rel_orient = 'conv'
            interaction_dist = other.length // 2 + other.intergene + (gene.length - 1)
    
        elif other.orientation == 0 and gene.orientation == 0: # --|--> |---->
            rel_orient = 'downstr'
            interaction_dist = other.length // 2 + other.intergene

        elif other.orientation == 1 and gene.orientation == 0: # <--|-- |---->
            rel_orient = 'div'
            interaction_dist = (other.length // 2 + 1) + other.intergene

        else:                                                  # <--|-- <----|
            rel_orient = 'upstr'
            interaction_dist = (other.length // 2 + 1) + other.intergene + (gene.length - 1)
            
        result_dict['gene_id'].append(gene.id)
        result_dict['gene_type'].append(gene_types[gene.gene_type])
        result_dict['gene_orient'].append(orient_name[gene.orientation])
        result_dict['other_id'].append(other.id)
        result_dict['other_type'].append(gene_types[other.gene_type])
        result_dict['other_orient'].append(orient_name[other.orientation])
        result_dict['rel_orient'].append(rel_orient)
        result_dict['intergene_dist'].append(other.intergene)
        result_dict['interaction_dist'].append(interaction_dist)
            
            
        #                                                       gene - other
        other = indiv.genes[(i_gene + 1) % indiv.nb_genes]
        if gene.orientation == 0 and other.orientation == 1:   # |----> <--|--
            rel_orient = 'conv'
            interaction_dist = gene.length + gene.intergene + (other.length // 2 - 1)
            
        elif gene.orientation == 0 and other.orientation == 0: # |----> --|-->
            rel_orient = 'upstr'
            interaction_dist = gene.length + gene.intergene + other.length // 2
    
        elif gene.orientation == 1 and other.orientation == 0: # <----| --|-->
            rel_orient = 'div'
            interaction_dist = (gene.intergene + 1) + other.length // 2
            
        else:                                                  # <----| <--|--
            rel_orient = 'downstr'
            interaction_dist = (gene.intergene + 1) + (other.length // 2 - 1)
                    
        result_dict['gene_id'].append(gene.id)
        result_dict['gene_type'].append(gene_types[gene.gene_type])
        result_dict['gene_orient'].append(orient_name[gene.orientation])
        result_dict['other_id'].append(other.id)
        result_dict['other_type'].append(gene_types[other.gene_type])
        result_dict['other_orient'].append(orient_name[other.orientation])
        result_dict['rel_orient'].append(rel_orient)
        result_dict['intergene_dist'].append(gene.intergene)
        result_dict['interaction_dist'].append(interaction_dist)

    return pd.DataFrame.from_dict(result_dict)

In [ ]:
def get_whole_stats(indiv):
    
    # Count all pairs of interacting genes
    
    gene_positions, genome_size = indiv.compute_gene_positions(include_coding=True)
    
    result_dict = {'gene_id': [],
                   'gene_type': [],
                   'gene_orient': [],
                   'other_id': [],
                   'other_type': [],
                   'other_orient': [],
                   'rel_orient': [],
                   'intergene_dist': [],
                   'interaction_dist': []}
        
    for i_gene, gene in enumerate(indiv.genes):
        for i_other, other in enumerate(indiv.genes):
            
            if i_other == i_gene:
                continue
            
            # As in evotsc.compute_inter_matrix(), the relevant distance is
            # the distance between the promoter of the focal gene and the 
            # middle of the other gene.
            #                                                       other  -  gene
                            
            
            if other.orientation == 0:  # Leading
                pos_other = gene_positions[i_other] + other.length // 2
            else:  # Lagging
                pos_other = gene_positions[i_other] - other.length // 2


            pos_1_minus_2 = gene_positions[i_gene] - pos_other
            pos_2_minus_1 = - pos_1_minus_2

            # We want to know whether gene i comes before or after gene j
            # Before: -------1--2-------- or -2---------------1-
            # After:  -------2--1-------- or -1---------------2-

            if pos_1_minus_2 < 0: # -------1--2-------- ou -1---------------2-
                if pos_2_minus_1 < genome_size + pos_1_minus_2: # -------1--2--------
                    distance = pos_2_minus_1
                    i_before_j = True
                else: # -1---------------2-
                    distance = genome_size + pos_1_minus_2
                    i_before_j = False

            else: # -------2--1-------- ou -2---------------1-
                if pos_1_minus_2 < genome_size + pos_2_minus_1: # -------2--1--------
                    distance = pos_1_minus_2
                    i_before_j = False
                else:
                    distance = genome_size + pos_2_minus_1
                    i_before_j = True
                    

            # Exit early if genes are too far
            if distance > indiv.interaction_dist:
                # inter_matrix[i, j] and inter_matrix[j, i] are already 0.0
                continue

            result_dict['gene_id'].append(gene.id)
            result_dict['gene_type'].append(gene_types[gene.gene_type])
            result_dict['gene_orient'].append(orient_name[gene.orientation])
            result_dict['other_id'].append(other.id)
            result_dict['other_type'].append(gene_types[other.gene_type])
            result_dict['other_orient'].append(orient_name[other.orientation])
            
            # we want to know if gene is ... of other
            if i_before_j:
                rel_orients = [['upstr', 'conv'],
                               ['div', 'downstr']]
            else:
                rel_orients = [['downstr', 'div'],
                               ['conv', 'upstr']]
            rel_orient = rel_orients[gene.orientation][other.orientation]
            
            result_dict['rel_orient'].append(rel_orient)
            

            if i_before_j:
                if gene.orientation == 0 and other.orientation == 0:
                    intergene_dist = distance - gene.length - other.length // 2
                elif gene.orientation == 0 and other.orientation == 1:
                    intergene_dist = distance - gene.length - (other.length // 2 - 1) 
                elif gene.orientation == 1 and other.orientation == 0:
                    intergene_dist = (distance - 1) - other.length // 2
                else: #gene.orientation == 1 and other.orientation == 1:
                    intergene_dist = (distance - 1) - (other.length // 2 - 1)
            
            else:
                if other.orientation == 0 and gene.orientation == 0:
                    intergene_dist = distance - other.length // 2
                elif other.orientation == 0 and gene.orientation == 1:
                    intergene_dist = distance - other.length // 2 - (gene.length - 1)
                elif other.orientation == 1 and gene.orientation == 0:
                    intergene_dist = distance - (other.length // 2 + 1)
                else: #other.orientation == 1 and gene.orientation == 1:
                    intergene_dist = distance - (other.length // 2 + 1) - (gene.length - 1)
            

            result_dict['intergene_dist'].append(intergene_dist)
            result_dict['interaction_dist'].append(distance)
            
    return pd.DataFrame.from_dict(result_dict)

In [ ]:
def get_full_pair_stats(exp_path, gen, only_pairs=True):
    rep_dirs = sorted([d for d in exp_path.iterdir() if (d.is_dir() and d.name.startswith("rep"))])
    
    full_stats = pd.DataFrame()
        
    for rep, rep_dir in enumerate(rep_dirs):
        
        best_indiv = evotsc_lib.get_best_indiv(rep_dir, gen)

        if only_pairs:
            indiv_stats = get_pair_stats(best_indiv)
        else:
            indiv_stats = get_whole_stats(best_indiv)
        indiv_stats.insert(0, 'Replicate', rep)
        full_stats = pd.concat([full_stats, indiv_stats])
            
    return full_stats

In [ ]:
pair_stats = get_full_pair_stats(exp_path, gen)

In [ ]:
whole_stats = get_full_pair_stats(exp_path, gen, only_pairs=False)

## Plot the average number and distance of gene pairs per gene type

In [ ]:
def plot_gene_stats(stats, plot_counts, highlight, name):
    
    # If plot_counts is true, the bars represent the count for each pair type and
    # we annotate them with the average distances; if count_bars is false, we 
    # plot the opposite: the bars represent average distances, and are annotated
    # with the count for each pair type.
    

    fig, axs = plt.subplots(3, 3, sharey='all', figsize=(12, 12), dpi=dpi)
    
    
    ## Massage the data into usable form
    group_cols = ['gene_type', 'other_type', 'rel_orient']
    group_dims = [gene_types, gene_types, rel_orients]
    group_cols_rep = [*group_cols, 'Replicate']
    group_dims_rep = [*group_dims, range(nb_reps)]
    idx = pd.MultiIndex.from_product(group_dims, names=group_cols)
    idx_rep = pd.MultiIndex.from_product(group_dims_rep, names=group_cols_rep)
    
    # Average number of pairs of each type (characterized by the values from the `group_cols` columns)
    pair_numbers = stats.groupby(group_cols).count().reindex(idx, fill_value=0).sort_index()['gene_id'] / nb_reps
    pair_numbers_rep = stats.groupby(group_cols_rep).count().reindex(idx_rep, fill_value=0).sort_index()['gene_id'] 

    
    # Average distance between the genes in the pairs (global average over the pairs from all individuals)
    pair_distances = stats.groupby(group_cols).mean().reindex(idx, fill_value=0).sort_index()['intergene_dist']
    pair_distances_rep = stats.groupby(group_cols_rep).mean().reindex(idx_rep, fill_value=0).sort_index()['intergene_dist']
        
    # Average distances are computed correctly, even when some replicates do not have a
    # specific kind of pair present (they are not counted as a 0 in the average).
    
    
    base_colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red']
    
    nb_orient = len(rel_orients)
    x_bar = np.arange(nb_orient)
    
    ## Plot
    for i_gene_type, gene_type in enumerate(gene_types):
        for i_other_type, other_type in enumerate(gene_types):
                
            ax = axs[i_gene_type][i_other_type]    
            
            # Plot the replicate-averaged data
            if plot_counts:
                bar_data = pair_numbers.loc[(gene_type, other_type)]
                rep_data = pair_numbers_rep.loc[(gene_type, other_type)]

            else: # plot distances
                bar_data = pair_distances.loc[(gene_type, other_type)]
                rep_data = pair_distances_rep.loc[(gene_type, other_type)]
                
            alphas = np.ones(len(base_colors))
            
            if highlight:
                alphas = np.ones(len(base_colors)) * 0.5
                if gene_type == other_type:
                    alphas[1] = 1 # highlight divergent pairs of the same type
                elif (gene_type == 'A' and other_type == 'B') or (gene_type == 'B' and other_type == 'A'):
                    alphas[0] = 1 # highlight convergent A/B or B/A pairs
    
            colors = [mpl.colors.to_rgba(base_colors[i], alphas[i]) for i in range(len(alphas))]

                
            rects = ax.bar(x_bar, bar_data, color=colors)
            
            # Plot the corresponding boxplot, or individual replicates            
            if nb_reps > 10: # If we have many replicates, draw a boxplot over the means
                rep_data = rep_data.unstack(level='rel_orient')
                ax.boxplot(rep_data, positions=x_bar, manage_ticks=False, medianprops={'color':'black'})

            else: # Else, just draw every replicate.
                rep_data = rep_data.unstack(level='Replicate')
                bar_width = 0.8 # default value
                x_rep = (np.tile(x_bar, (nb_reps, 1)).T +
                         np.tile(np.linspace(-bar_width/2, bar_width/2, nb_reps+2)[1:-1], (nb_orient, 1)))

                if plot_counts:
                    ax.plot(x_rep, rep_data,
                            marker='o', linestyle='', markeredgecolor='black', markerfacecolor='none')
                # If we're plotting distances, we don't consider pairs that are not present when computing
                # the average distance, so we use a different marker to show that
                else:
                    rep_data_nonzero = rep_data.where(rep_data > 0)
                    rep_data_zero = rep_data.where(rep_data == 0)
                    ax.plot(x_rep, rep_data_nonzero,
                            marker='o', linestyle='', markeredgecolor='black', markerfacecolor='none')
                    ax.plot(x_rep, rep_data_zero,
                            marker='x', linestyle='', markeredgecolor='black', markerfacecolor='none')


            # Plot setup
            #if plot_counts:
            #    ax.set_ylim(0, 32)

            ax.set_xticks(x_bar)
            ax.set_xticklabels(labels=[orient + '.' for orient in rel_orients], fontsize=16)
            ax.yaxis.set_tick_params(labelsize=16)
            
            ax.grid(axis='y', linestyle=':')

            if i_gene_type == 0: # First line
                ax.set_title(fr"$\rightarrow$ {other_type}", fontsize=22)
            
            if i_other_type == 0: # First column
                ax.set_ylabel(fr"{gene_type} $\rightarrow$", rotation='vertical', fontsize=22)

    fig.suptitle('Gene pair counts by gene type and relative orientation', size=24, x=0.528)
    plt.tight_layout()
                        
    plt.savefig(exp_path.joinpath(f'{name}.pdf'), dpi=dpi, bbox_inches='tight')
    plt.show()

In [ ]:
plot_gene_stats(pair_stats, plot_counts=True, highlight=False, name='gene_pair_counts')

In [ ]:
def print_pair_stats(stats):
    group_cols = ['gene_type', 'other_type', 'rel_orient']
    group_dims = [gene_types, gene_types, rel_orients]
    idx = pd.MultiIndex.from_product(group_dims, names=group_cols)
    pairs_avg = (stats.groupby(group_cols).count()
                      .reindex(idx, fill_value=0)
                      .sort_index()['gene_id'] / nb_reps)
    
    pair_distances = (stats.groupby(group_cols).mean()
                           .reindex(idx, fill_value=0)
                           .sort_index()['intergene_dist'])
    
    for gene_type in gene_types:
    
        total_pairs = pairs_avg.loc[(gene_type)].sum()
        div_pairs = pairs_avg.loc[(gene_type, gene_type, 'div')]
        dist_pairs = pair_distances.loc[(gene_type, gene_type, 'div')]

        print(f"Pairs of divergent {gene_type} genes: {div_pairs:.3} / {total_pairs:.4} ({div_pairs / total_pairs * 100:.3} %)")#, mean distance: {dist_pairs:.5}") 
        
    print()    
    
    for other_type in gene_types:
        for gene_type in gene_types:
            
            total_pairs = pairs_avg.loc[(gene_type)].sum()
            conv_pairs = pairs_avg.loc[(gene_type, other_type, 'conv')]
            
            print(f'{gene_type:2} genes in a convergent {other_type:2}/{gene_type:2} pair: {conv_pairs:.3} / {total_pairs / 2:.3} ({conv_pairs / (total_pairs / 2) * 100:.3} %)')
            
    print()
    
    print(f"Legend example: {pairs_avg.loc[('AB', 'B', 'upstr')]} AB genes are upstream of B genes.")
    
    print()
    
    for other_type in gene_types:
        for gene_type in gene_types:
            total_pairs = pairs_avg.loc[(gene_type, other_type)].sum()
            print(f'Total number of {gene_type:2}/{other_type:2} pairs: {total_pairs:.4}')

In [ ]:
print_pair_stats(pair_stats)

## Plot the positive / negative supercoiling due to each gene type in each environment

In [ ]:
def get_pos_neg_supercoiling(exp_path, gen, pairs_only=False):
        
    result_dict = {'Replicate': [],
                   'env': [],
                   'gene_type': [],
                   'other_type': [],
                   'delta_sc': [],
                   'sign': [] }
    
    nb_genes = int(params['nb_genes'])
    genes_per_type = nb_genes / len(gene_types)
    
    envs = ['A', 'B']
    signs = ['neg', 'pos']
    
    for rep, rep_dir in enumerate(rep_dirs):
        
        best_indiv = evotsc_lib.get_best_indiv(rep_dir, gen)
        
        inter_matrix = best_indiv.compute_inter_matrix()
        
        for env in envs:
        
            sigma_env = params[f'sigma_{env}']
            final_expr = best_indiv.run_system(sigma_env)[-1, :]
        
            for j_gene, other in enumerate(best_indiv.genes):
                for i_gene, gene in enumerate(best_indiv.genes):
                    
                    if (not pairs_only or
                        ((i_gene == (j_gene + 1) % nb_genes or
                        i_gene == (j_gene + nb_genes - 1) % nb_genes))):
                        
                            delta_sc = inter_matrix[i_gene][j_gene] * final_expr[j_gene]
                            
                            result_dict['Replicate'].append(rep)
                            result_dict['env'].append(env)
                            result_dict['gene_type'].append(gene_types[gene.gene_type])
                            result_dict['other_type'].append(gene_types[other.gene_type])
                            result_dict['delta_sc'].append(delta_sc)
                            result_dict['sign'].append(signs[bool(delta_sc > 0)])
                            
    # DataFrame trickery: reindex from a complete MultiIndex to add missing lines
    col_names = ['gene_type', 'other_type', 'env', 'sign', 'Replicate']
    col_dims = [gene_types, gene_types, envs, signs, range(nb_reps)]
    idx = pd.MultiIndex.from_product(col_dims, names=col_names)
    
    # Get the value per replicate (averaged over the genes of that type)
    df = pd.DataFrame.from_dict(result_dict)
    summed_df = df.groupby(col_names).sum() / genes_per_type
    
    # Reindex to add the missing cases
    reindexed_df = summed_df.reindex(idx, fill_value=0).sort_index()

    return reindexed_df

In [ ]:
pos_neg_stats_pairs = get_pos_neg_supercoiling(exp_path, gen, pairs_only=True)

In [ ]:
def plot_pos_neg_supercoiling(data, plot_name):
    
    fig, axs = plt.subplots(3, 3, sharey=True, figsize=(12, 12), dpi=dpi)
    
    x = [0, 1]
    
    rep_avg = data.groupby(['gene_type', 'other_type', 'env', 'sign']).mean()

    # Other: the gene that **creates** the SC delta -> lines
    # Gene: the gene that **experiences** the SC delta -> columns
    for i_other_type, other_type in enumerate(gene_types):
        for i_gene_type, gene_type in enumerate(gene_types):
            
            ax = axs[i_other_type][i_gene_type]
            
            # Rects
            pos_bars = [rep_avg.loc[(gene_type, other_type, 'A', 'pos')]['delta_sc'],
                        rep_avg.loc[(gene_type, other_type, 'B', 'pos')]['delta_sc']]
            neg_bars = [rep_avg.loc[(gene_type, other_type, 'A', 'neg')]['delta_sc'],
                        rep_avg.loc[(gene_type, other_type, 'B', 'neg')]['delta_sc']]
            
            width = 0.7
            pos_rects = ax.bar(x, pos_bars, color='tab:red', width=width)
            neg_rects = ax.bar(x, neg_bars, color='tab:green', width=width)
            
            # Boxplots
            ax.boxplot(data.unstack(level='env').loc[(gene_type, other_type, 'pos')], positions=x,
                       manage_ticks=False, medianprops={'color':'black'})
            ax.boxplot(data.unstack(level='env').loc[(gene_type, other_type, 'neg')], positions=x,
                       manage_ticks=False, medianprops={'color':'black'})

            ax.set_xticks(x)
            ax.set_xlim(-0.75 * width, 1 + 0.75*width)
            ax.set_xticklabels(labels=["Env. A", "Env. B"], fontsize=16)
            ax.yaxis.set_tick_params(labelsize=16)
            
            ax.grid(axis='y', linestyle=':')
            ax.invert_yaxis()

            if i_other_type == 0: # First line
                ax.set_title(fr"$\rightarrow$ {gene_type}", fontsize=22)
            if i_gene_type == 0:
                ax.set_ylabel(fr"{other_type} $\rightarrow$", rotation='vertical', fontsize=22)
                
                
    fig.suptitle('Transcription-generated supercoiling by gene type and environment', size=24, x=0.528)
    plt.tight_layout()
    
    plt.savefig(exp_path.joinpath(plot_name), dpi=dpi, bbox_inches='tight')
    plt.show()

In [ ]:
plot_pos_neg_supercoiling(pos_neg_stats_pairs, 'pos_neg_supercoiling_pairs.pdf')

In [ ]:
def print_total_sc_values(pos_neg_stats):
    # Average by replicate
    avg_stats = pos_neg_stats.groupby(['gene_type', 'other_type', 'env', 'sign']).mean()
    
    # Sum positive and negative SC
    sum_stats = avg_stats.groupby(['gene_type', 'other_type', 'env']).sum()
    
    for other_type in gene_types:
        for env in ['A', 'B']:
            print(f"SC contribution of {other_type:2} genes on {other_type:2} genes env {env}: {sum_stats.loc[(other_type, other_type, env)]['delta_sc']:.3}")

In [ ]:
print_total_sc_values(pos_neg_stats_pairs)

# Regulatory sub-networks

## Generate sub-networks of _k_ consecutive genes

In [ ]:
def extract_subnetwork(indiv, i_start, size, keep_ids=False):
    
    orig_pos, indiv_len = indiv.compute_gene_positions(include_coding=True)
    
    new_genes = []
    for i_gene in range(i_start, i_start + size):
        new_gene = indiv.genes[(i_gene % indiv.nb_genes)].clone()
        if keep_ids == False:
            new_gene.id = i_gene % indiv.nb_genes
        new_genes.append(new_gene)


    clone = evotsc.Individual(genes=new_genes,
                              interaction_dist=indiv.interaction_dist,
                              interaction_coef=indiv.interaction_coef,
                              sigma_basal=indiv.sigma_basal,
                              sigma_opt=indiv.sigma_opt,
                              epsilon=indiv.epsilon,
                              m=indiv.m,
                              selection_coef=indiv.selection_coef,
                              rng=indiv.rng)

    _, clone_len = clone.compute_gene_positions(include_coding=True)

    new_genes[-1].intergene += indiv_len - clone_len

    return clone

In [ ]:
def plot_subnetworks(rep, gen, networks):
    
    indiv = evotsc_lib.get_best_indiv(exp_path.joinpath(f'rep{rep:02}'), gen)
    
    orig_pos, indiv_len = indiv.compute_gene_positions(include_coding=True)
    
    sub_path = exp_path.joinpath(f'sub_rep{rep:02}')
    sub_path.mkdir(exist_ok=True)

    for i_network, (network_size, i_start) in enumerate(networks):
        clone = extract_subnetwork(indiv, i_start, network_size)

        # For plotting at the same position as the original individual
        shift = orig_pos[i_start]
        if indiv.genes[i_start].orientation == 1: # Lagging
            shift -= (indiv.genes[i_start].length - 1)

        for env in ['A', 'B']:
            sigma = params[f'sigma_{env}']
            name = sub_path.joinpath(f'sub_{network_size}_genes_{i_start:02}_env_{env}.png')
            
            evotsc_plot.plot_genome_and_tsc(clone, sigma=sigma, coloring_type='on-off',
                                            show_bar=(i_network % 2 == 0),
                                            print_ids=True,
                                            id_central=(network_size-1)/2,  # Only plot the central gene
                                            naming_type='id',
                                            shift=-shift, plot_name=name)   

In [ ]:
if figure_rep < nb_reps:
    plot_subnetworks(rep=figure_rep, gen=gen, networks=[(3, 30), (5, 29), (7, 3), (9, 2)])

## Statistics of the subnetworks: final expression of each gene and fitness

In [ ]:
def compute_subnetwork_fitness(indiv):
    # Take into account the fact that the individual has k_A, k_B, k_AB < 20 genes of each type
    
    expr_levels_A = indiv.run_system(params['sigma_A'])
    expr_levels_B = indiv.run_system(params['sigma_B'])
    
    # The minimal expression level is exp(-m)
    target_A = np.array([1.0, 1.0, np.exp(-indiv.m)]) # Gene types are AB, A, B
    target_B = np.array([1.0, np.exp(-indiv.m), 1.0])

    
    nb_genes_per_type = np.array([0, 0, 0], dtype=int)
    
    for gene in indiv.genes:
        nb_genes_per_type[gene.gene_type] += 1
        
    for i_type in range(len(gene_types)):
        if nb_genes_per_type[i_type] == 0:
            nb_genes_per_type[i_type] = 1  # No genes of that type: expr should be 0, but divide by 1
            target_A[i_type] = target_B[i_type] = 0 # Also don't count this type in the fitness

    # Environment A
    gene_expr_A = np.zeros(3)
    for i_gene, gene in enumerate(indiv.genes):
        gene_expr_A[gene.gene_type] += expr_levels_A[-1, i_gene]


    gap_A = np.square(gene_expr_A / nb_genes_per_type - target_A).sum()

    # Environment B
    gene_expr_B = np.zeros(3)
    for i_gene, gene in enumerate(indiv.genes):
        gene_expr_B[gene.gene_type] += expr_levels_B[-1, i_gene]


    gap_B = np.square(gene_expr_B / nb_genes_per_type - target_B).sum()

    fitness = np.exp(- indiv.selection_coef * (gap_A + gap_B))

    return fitness

In [ ]:
def compute_subnetwork_stats(exp_path, gen, network_sizes):
    
    rep_dirs = sorted([d for d in exp_path.iterdir() if (d.is_dir() and d.name.startswith("rep"))])
    
    full_stats = pd.DataFrame()
        
    for rep, rep_dir in enumerate(rep_dirs):
        
        indiv = evotsc_lib.get_best_indiv(rep_dir, gen)
        
        result_dict = {'Replicate': [],
               'network_size': [],
               'start_pos': [],
               'subnetwork_pos': [],
               'gene_id': [],
               'gene_type': [],
               'gene_orient': [],
               'final_expr_A': [],
               'final_expr_B': [],
               'fitness': []}
    
        for size in network_sizes:

            for i_start in range(indiv.nb_genes):
                sub_indiv = extract_subnetwork(indiv, i_start, size, keep_ids=True)

                (expr_A, expr_B), _ = sub_indiv.evaluate(params['sigma_A'], params['sigma_B'])
                fitness = compute_subnetwork_fitness(sub_indiv)

                for i_gene, gene in enumerate(sub_indiv.genes):
                    result_dict['Replicate'].append(rep)
                    result_dict['network_size'].append(size)
                    result_dict['start_pos'].append(i_start)
                    result_dict['subnetwork_pos'].append(i_gene)
                    result_dict['gene_id'].append(gene.id)
                    result_dict['gene_type'].append(gene_types[gene.gene_type])
                    result_dict['gene_orient'].append(orient_name[gene.orientation])
                    result_dict['final_expr_A'].append(expr_A[-1, i_gene])
                    result_dict['final_expr_B'].append(expr_B[-1, i_gene])
                    result_dict['fitness'].append(fitness)


        indiv_stats = pd.DataFrame.from_dict(result_dict)

        full_stats = pd.concat([full_stats, indiv_stats])
            
    return full_stats

In [ ]:
# Full stats
stats = compute_subnetwork_stats(exp_path, gen, network_sizes=range(1, int(params['nb_genes'])+1))

In [ ]:
# Statistics for the genes at the middle position of a subnetwork (of odd sizes only)
central_gene_stats = pd.concat(stats[(stats["network_size"] == network_size) &
                                     (stats["subnetwork_pos"] == network_size // 2)]
                               for network_size in np.arange(1, 60, 2)).copy()

## Minimum network size for the central gene to regain its function

In [ ]:
def compute_min_subnetwork_size(stats, central_gene_stats):
    
    half_dist = (1 - np.exp(- params['m'])) / 2 # Distance between min_expr and half_expr or half_expr and 1
    nb_genes = stats['gene_id'].nunique()
    
    whole_stats = stats[stats['network_size'] == 60]

    data = {'A': {'A': [], 'B': []},
            'B': {'A': [], 'B': []},
            'AB': {'A': [], 'B': []}}
    
    res_dict = {'Replicate': [],
                'gene_id': [],
                'gene_pos': [], # position in the gene list
                'gene_type': [],
                'min_size_A': [],
                'min_size_B': []}
    
    target_A = {'A': 1, 'B': np.exp(- params['m']), 'AB': 1}
    target_B = {'A': np.exp(- params['m']), 'B': 1, 'AB': 1}
    
    for rep in range(nb_reps):
        rep_stats = whole_stats[whole_stats['Replicate'] == rep]
        for gene_id in range(nb_genes):
            
            gene_stats = rep_stats[rep_stats['gene_id'] == gene_id].iloc[0]
            cur_gene_type = gene_stats['gene_type']
            
            for gene_type in gene_types:
                if cur_gene_type == gene_type:
                    if (np.abs(gene_stats['final_expr_A'] - target_A[gene_type]) < half_dist and
                        np.abs(gene_stats['final_expr_B'] - target_B[gene_type]) < half_dist):

                        gene_stats = central_gene_stats[(central_gene_stats['Replicate'] == rep) &
                                                        (central_gene_stats['gene_id'] == gene_id)]

                        for min_size_A in np.arange(1, nb_genes+1, step=2):
                            size_data = gene_stats[gene_stats['network_size'] == min_size_A].iloc[0]
                            if np.abs(size_data['final_expr_A'] - target_A[gene_type]) < half_dist:
                                break

                        for min_size_B in np.arange(1, nb_genes+1, step=2):
                            size_data = gene_stats[gene_stats['network_size'] == min_size_B].iloc[0]
                            if np.abs(size_data['final_expr_B'] - target_B[gene_type]) < half_dist:
                                break

                        res_dict['Replicate'].append(rep)
                        res_dict['gene_id'].append(gene_id)
                        
                        gene_pos = (gene_stats.iloc[0]['start_pos'] + gene_stats.iloc[0]['subnetwork_pos']) % nb_genes
                        res_dict['gene_pos'].append(gene_pos)
                        res_dict['gene_type'].append(gene_type)
                        res_dict['min_size_A'].append(min_size_A)
                        res_dict['min_size_B'].append(min_size_B)
                        
    return pd.DataFrame.from_dict(res_dict)

In [ ]:
data = compute_min_subnetwork_size(stats, central_gene_stats)

In [ ]:
data.groupby('gene_type').count()

In [ ]:
def plot_min_needed_size(data):
    fig, ax = plt.subplots(figsize=(8, 5), dpi=dpi)
    ax.set_ylim(0, 30)
    ax.grid(linestyle=':', axis='y')
    
    delta = 2.5
    
    x_pos = [0, 1, delta, delta + 1, 2*delta, 2*delta + 1]
        
    ax.set_xticks(x_pos, labels=[f'{gene_type} env. {env}' for gene_type, env in
                                 itertools.product(gene_types, ['A', 'B'])])
    
    for i_gene_type, gene_type in enumerate(gene_types):
        violin = ax.violinplot(data[data['gene_type'] == gene_type][['min_size_A', 'min_size_B']],
                               positions=[delta*i_gene_type, delta*i_gene_type+1])
        
        for pc in violin['bodies']:
            pc.set_facecolor(gene_type_color[i_gene_type])

        for partname in ('cbars','cmins','cmaxes'):
            vp = violin[partname]
            vp.set_edgecolor(gene_type_color[i_gene_type])
            #vp.set_linewidth(1)

    for i_gene_type, gene_type in enumerate(gene_types):
        ax.boxplot(data[data['gene_type'] == gene_type][['min_size_A', 'min_size_B']],
                   positions=[delta*i_gene_type, delta*i_gene_type+1], showmeans=True,
                   manage_ticks=False, medianprops={'color':'black'},
                   meanprops={'marker':'_', 'markerfacecolor':'black', 'markeredgecolor':'black'})
    
    #ax.yaxis.set_major_locator(mpl.ticker.MultipleLocator(2))
    ax.set_yticks(np.arange(1, 31, step=4))
    ax.set_ylim(-1, 31)
    
    ax.set_ylabel('Minimum network size to reach activation target')
    
    plt.savefig(exp_path.joinpath('min_network_size.pdf'), dpi=dpi, bbox_inches='tight')

In [ ]:
plot_min_needed_size(data)

# Gene knockouts

## Arrange Individual and run_system() to accommodate a knocked-out gene

In [ ]:
@jit(nopython=True)
def run_system_numba_ko(nb_genes: int,
                        init_expr: np.ndarray,
                        inter_matrix: np.ndarray,
                        sigma_basal: float,
                        sigma_opt: float,
                        epsilon: float,
                        m: float,
                        sigma_env: float,
                        id_ko: int) -> np.ndarray:

    step_size = 0.5
    stop_dist = 1e-7
    max_eval_steps = 200

    temporal_expr = np.zeros((max_eval_steps+1, nb_genes))

    # Initial values at t = 0
    temporal_expr[0, :] = init_expr
    temporal_expr[0, id_ko] = 0.0


    # Iterate the system
    it = 1
    cont = True
    while cont:
        prev_expr = temporal_expr[it-1, :]
        sigma_local = inter_matrix @ prev_expr
        sigma_total = sigma_basal + sigma_local + sigma_env

        promoter_activity = 1.0 / (1.0 + np.exp((sigma_total - sigma_opt)/epsilon))

        # We subtract 1 to rescale between exp(-m) and 1
        iter_expr = np.exp(m * (promoter_activity - 1.0))

        nouv_expr = step_size * iter_expr + (1 - step_size) * prev_expr

        # Knockout
        nouv_expr[id_ko] = 0

        temporal_expr[it, :] = nouv_expr

        # Check if we're done
        dist = np.abs(nouv_expr - prev_expr).sum() / nb_genes

        prev_expr = nouv_expr

        if dist < stop_dist:
            cont = False

        if it == max_eval_steps:
            cont = False
        it += 1

    temporal_expr = temporal_expr[:it, :]

    return temporal_expr

In [ ]:
class KoIndividual(evotsc.Individual):
    def __init__(self,
                 orig_indiv: evotsc.Individual,
                 id_ko: int) -> None:
        
        super().__init__([gene.clone() for gene in orig_indiv.genes],
                         orig_indiv.interaction_dist,
                         orig_indiv.interaction_coef,
                         orig_indiv.sigma_basal,
                         orig_indiv.sigma_opt,
                         orig_indiv.epsilon,
                         orig_indiv.m,
                         orig_indiv.selection_coef,
                         orig_indiv.rng)
        
        self.id_ko = id_ko
                    

    def run_system(self, sigma_env):

        init_expr = np.array([gene.basal_expression for gene in self.genes])

        self.inter_matrix = self.compute_inter_matrix()

        return run_system_numba_ko(nb_genes=self.nb_genes,
                                   init_expr=init_expr,
                                   inter_matrix=self.inter_matrix,
                                   sigma_basal=self.sigma_basal,
                                   sigma_opt=self.sigma_opt,
                                   epsilon=self.epsilon,
                                   m=self.m,
                                   sigma_env=sigma_env,
                                   id_ko=self.id_ko)

## Plot genome and TSC with one knocked-out gene

In [ ]:
def plot_all_kos(rep, gen):
    
    rep_dir = exp_path.joinpath(f'rep{rep}')
    
    ko_path = exp_path.joinpath(f'kos_rep{rep:02}')
    ko_path.mkdir(exist_ok=True)
    
    init_indiv = evotsc_lib.get_best_indiv(rep_dir, gen=gen)
    n = 100  # the number of data points
    _, genome_length = init_indiv.compute_gene_positions(include_coding=True)
    data_positions = np.linspace(0, genome_length, n, dtype=int)

    # Note: we do the difference between the supercoiling values of two genomes of the same individual,
    # so sigma_basal and sigma_env cancel out
    init_data = {'A': init_indiv.compute_final_sc_at(params['sigma_A'], data_positions),
                 'B': init_indiv.compute_final_sc_at(params['sigma_B'], data_positions)}
    
    init_expr = {'A': init_indiv.run_system(params['sigma_A'])[-1, :],
                 'B': init_indiv.run_system(params['sigma_B'])[-1, :]}
    
    
    half_expr = (1 + np.exp(-init_indiv.m)) / 2
    init_activ = {env: init_expr[env] > half_expr for env in envs}
    
    for i_ko in range(36, 37): 
    #for i_ko in range(init_indiv.nb_genes):
        
        ko_indiv = KoIndividual(orig_indiv=init_indiv, id_ko=i_ko)
                                            
        for env in envs:
                                            
            ko_data = ko_indiv.compute_final_sc_at(params[f'sigma_{env}'], data_positions)
            
            final_expr = ko_indiv.run_system(params[f'sigma_{env}'])[-1, :]
            ko_activ = final_expr > half_expr
            
            switched_genes = init_activ[env] != ko_activ
            
            show_bar = env == 'A'
            
            plot_name = ko_path.joinpath(f'ko_genome_and_tsc_env_{env}_gene_{i_ko:02}.png')
            
            evotsc_plot.plot_genome_and_tsc(ko_indiv, sigma=params[f'sigma_{env}'], coloring_type='on-off',
                                            hatched_genes=switched_genes,
                                            print_ids=True, ring_data=np.abs(init_data[env]-ko_data), 
                                            ring_color_type='delta', 
                                            show_bar=show_bar, bar_text='$|\Delta\sigma_{TSC}|$',
                                            id_interval=5, id_ko=i_ko,
                                            plot_name=plot_name)

In [ ]:
if figure_rep < nb_reps:
    plot_all_kos(rep=figure_rep, gen=gen)

## Compute the effective graph: genes switched on or off by a KO

In [ ]:
def get_effective_graph(best_indiv, sigma):
    
    _, genome_length = best_indiv.compute_gene_positions(include_coding=True)
    
    init_expr = best_indiv.run_system(sigma)[-1, :]
    half_expr = (1 + np.exp(-best_indiv.m)) / 2
    init_activ = init_expr > half_expr
    

    ## Build the graph
    inter_graph = nx.DiGraph()

    # Nodes
    for i_gene, gene in enumerate(best_indiv.genes):
        inter_graph.add_node(i_gene, gene=gene)

    ## Compute the interactions: if KOing gene A switches gene B on or off, add A -> B arrow to the graph

    for i_ko in range(best_indiv.nb_genes):
        
        ko_indiv = KoIndividual(orig_indiv=best_indiv, id_ko=i_ko)
                                                                    
        final_expr = ko_indiv.run_system(sigma)[-1, :]
        ko_activ = final_expr > half_expr
        
        for i_other in range(best_indiv.nb_genes):
            if i_other == i_ko:
                continue
            
            if init_activ[i_other] != ko_activ[i_other]:
                if init_activ[i_other]: # KOing i_ko inhibits i_other: -> i_ko activates i_other
                    inter_graph.add_edge(i_ko, i_other, kind='activ')
                else:
                    inter_graph.add_edge(i_ko, i_other, kind='inhib')

    return best_indiv, inter_graph

In [ ]:
def get_all_effective_graphs(exp_path, gen):
    rep_dirs = sorted([d for d in exp_path.iterdir() if (d.is_dir() and d.name.startswith("rep"))])
    
    graphs = {rep: {} for rep in range(len(rep_dirs))}
    
    for rep, rep_dir in enumerate(rep_dirs):
        best_indiv = evotsc_lib.get_best_indiv(rep_path=rep_dir, gen=gen)
        for env in ['A', 'B']:
            graphs[rep][env] = get_effective_graph(best_indiv, params[f'sigma_{env}'])
    
    return graphs

In [ ]:
effective_graphs = get_all_effective_graphs(exp_path, gen)

## Plot the effective graphs

In [ ]:
def plot_effective_graph(indiv, inter_graph, method, plot_name=None):
    
    ## Draw the figure
    plt.figure(figsize=(12, 12), dpi=dpi)
    plt.box(False)

    # Choose the layout
    if method == 'circular':
        layout = nx.circular_layout(inter_graph)
        for node in layout:
            x, y = layout[node]
            layout[node] = y, x # symmetry along the y=x axis to match other graphs

    elif method == 'graphviz':
        layout = nx.nx_agraph.graphviz_layout(inter_graph)
        for node in layout:
            x, y = layout[node]
            layout[node] = -x, -y # 180 degree rotation


    # Draw the nodes
    node_size = 400
    nx.draw_networkx_nodes(inter_graph, layout, node_size=node_size,
                           node_color=[gene_type_color[gene.gene_type] for gene in indiv.genes])
    nx.draw_networkx_labels(inter_graph, layout)

    # Draw the edges
    activ_edges = [e for e in inter_graph.edges if inter_graph[e[0]][e[1]]['kind'] == 'activ'] 
    inhib_edges = [e for e in inter_graph.edges if inter_graph[e[0]][e[1]]['kind'] == 'inhib']

    inhib_arrow = mpl.patches.ArrowStyle.CurveFilledB(head_length=0, head_width=0.5)

    nx.draw_networkx_edges(inter_graph, layout, edgelist=activ_edges, node_size=node_size,
                           edge_color='tab:green', connectionstyle='arc3,rad=0.1')
    nx.draw_networkx_edges(inter_graph, layout, edgelist=inhib_edges, node_size=node_size+75,
                           arrowstyle=inhib_arrow, edge_color='tab:red',
                           connectionstyle='arc3,rad=0.1')

    if plot_name:
        plt.savefig(plot_name, bbox_inches='tight', dpi=dpi)
    plt.show()
    plt.close()

In [ ]:
def plot_combined_effective_graphs(effective_graphs, method, is_random=False):
    graph_path = exp_path.joinpath(f'effective_graphs')
    graph_path.mkdir(exist_ok=True)
    
    for rep in effective_graphs.keys():
        indiv, graph_A = effective_graphs[rep]['A']
        indiv, graph_B = effective_graphs[rep]['B']
        
        combined_graph = nx.compose(graph_A, graph_B)
        
        plot_name = f'combined_effective_graph_rep{rep:02}_{method}.pdf'
        if is_random:
            plot_name = f'rand_{plot_name}'

        full_name = graph_path.joinpath(plot_name)
        
        plot_effective_graph(indiv, combined_graph, method, full_name)

In [ ]:
if figure_rep < nb_reps:
    plot_combined_effective_graphs({rep: effective_graphs[rep] for rep in [figure_rep]},
                                   'graphviz', is_random=False)

## Make random individuals for comparison

In [ ]:
def make_random_effective_graphs():
    
    graphs = {rep: {} for rep in range(nb_reps)}
        
    for i_rep, rand_indiv in enumerate(rand_indivs):        
        for env in ['A', 'B']:
            graphs[i_rep][env] = get_effective_graph(rand_indivs[i_rep], params[f'sigma_{env}'])
    
    return graphs

In [ ]:
rand_effective_graphs = make_random_effective_graphs()

## Connected components

In [ ]:
def count_largest_wcc(effective_graphs):
    
    largest_wcc = np.zeros(len(effective_graphs))
    
    for rep in list(effective_graphs.keys()):
        indiv, graph_A = effective_graphs[rep]['A']
        indiv, graph_B = effective_graphs[rep]['B']

        combined_graph = nx.compose(graph_A, graph_B)
        
        largest_wcc[rep] = len(max(nx.weakly_connected_components(combined_graph), key=len))
        
    return largest_wcc

In [ ]:
def gather_all_wccs(effective_graphs):
    wccs = []
    
    for rep in list(effective_graphs.keys()):
        indiv, graph_A = effective_graphs[rep]['A']
        indiv, graph_B = effective_graphs[rep]['B']

        combined_graph = nx.compose(graph_A, graph_B)
        
        for wcc in nx.weakly_connected_components(combined_graph):
            wccs.append(len(wcc))
                
    return np.array(wccs)

In [ ]:
def plot_all_wccs(effective_graphs, rand_effective_graphs):

    evolved_wccs = gather_all_wccs(effective_graphs)

    rand_wccs = gather_all_wccs(rand_effective_graphs)

    plt.figure(figsize=(3, 7), dpi=dpi)
    plt.boxplot([evolved_wccs, rand_wccs],
                #showmeans=True,
                #meanprops={'marker':'_', 'markerfacecolor':'black', 'markeredgecolor':'black'},
                medianprops={'color':'black'})
    plt.xticks(ticks=[1, 2], labels=['Evolved', 'Random'])
    plt.ylim(-3, 63)
    plt.ylabel('Number of genes in the WCC')
    plt.grid(linestyle=':', axis='y')
    plt.savefig(exp_path.joinpath('effective_graph_wcc_distr.pdf'), bbox_inches='tight', dpi=dpi)
    plt.show()

In [ ]:
plot_all_wccs(effective_graphs, rand_effective_graphs)

### Show the exact count for every WCC size

In [ ]:
np.unique(gather_all_wccs(effective_graphs), return_counts=True)

## Stats on the effective graphs: out degree (number of switches)

In [ ]:
def dist_in_genes(nb_genes, i, j):
    pos_1_minus_2 = i - j
    pos_2_minus_1 = - pos_1_minus_2

    # We want to know whether gene 1 comes before or after gene 2
    # Before: -------1--2-------- or -2---------------1-
    # After:  -------2--1-------- or -1---------------2-

    if pos_1_minus_2 < 0: # -------1--2-------- ou -1---------------2-
        if pos_2_minus_1 < nb_genes + pos_1_minus_2: # -------1--2--------
            distance = pos_2_minus_1
        else: # -1---------------2-
            distance = nb_genes + pos_1_minus_2

    else: # -------2--1-------- ou -2---------------1-
        if pos_1_minus_2 < nb_genes + pos_2_minus_1: # -------2--1--------
            distance = pos_1_minus_2
        else:
            distance = nb_genes + pos_2_minus_1

    return distance

In [ ]:
def get_effective_graph_stats(effective_graphs):
    res_dict = {'Replicate': [],
                'gene_type': [],
                'env': [],
                'out_deg': [],
                'avg_dist': [],
                'max_dist': []}

    for rep in effective_graphs.keys():
        for env in ['A', 'B']:
            indiv, graph = effective_graphs[rep][env]
            for i_gene, gene in enumerate(indiv.genes):
                res_dict['Replicate'].append(rep)
                res_dict['gene_type'].append(gene_types[gene.gene_type])
                res_dict['env'].append(env)
                res_dict['out_deg'].append(graph.out_degree[i_gene])
                
                if graph.out_degree[i_gene] > 0:
                    dists = np.zeros(len(graph[i_gene].keys()), dtype=int)                
                    for i_neighbor, neighbor in enumerate(graph[i_gene].keys()):
                        dists[i_neighbor] = dist_in_genes(indiv.nb_genes, i_gene, neighbor)

                    res_dict['avg_dist'].append(np.mean(dists))
                    res_dict['max_dist'].append(np.max(dists))
                
                else:
                    res_dict['avg_dist'].append(0)
                    res_dict['max_dist'].append(0)
                
    return pd.DataFrame.from_dict(res_dict)

In [ ]:
graph_stats = get_effective_graph_stats(effective_graphs)

In [ ]:
rand_graph_stats = get_effective_graph_stats(rand_effective_graphs)

In [ ]:
def get_combined_graph_stats(effective_graphs):
    res_dict = {'Replicate': [],
                'gene_type': [],
                'out_deg': [],
                'in_deg': [],
                'avg_dist': [],
                'max_dist': []}

    for rep in effective_graphs.keys():

        indiv, graph_A = effective_graphs[rep]['A']
        indiv, graph_B = effective_graphs[rep]['B']
        combined_graph = nx.compose(graph_A, graph_B)
        
        for i_gene, gene in enumerate(indiv.genes):
            res_dict['Replicate'].append(rep)
            res_dict['gene_type'].append(gene_types[gene.gene_type])
            res_dict['out_deg'].append(combined_graph.out_degree[i_gene])
            res_dict['in_deg'].append(combined_graph.in_degree[i_gene])

            if combined_graph.out_degree[i_gene] > 0:
                dists = np.zeros(len(combined_graph[i_gene].keys()), dtype=int)                
                for i_neighbor, neighbor in enumerate(combined_graph[i_gene].keys()):
                    dists[i_neighbor] = dist_in_genes(indiv.nb_genes, i_gene, neighbor)

                res_dict['avg_dist'].append(np.mean(dists))
                res_dict['max_dist'].append(np.max(dists))

            else:
                res_dict['avg_dist'].append(0)
                res_dict['max_dist'].append(0)
                
    return pd.DataFrame.from_dict(res_dict)

In [ ]:
combined_stats = get_combined_graph_stats(effective_graphs)

In [ ]:
combined_rand_stats = get_combined_graph_stats(rand_effective_graphs)

## Plot the average out degree of nodes in the network

In [ ]:
def plot_degree_by_env_combined(graph_stats, rand_graph_stats, deg_type):
    
    fig, ax = plt.subplots(figsize=(8, 5), dpi=dpi)

    mean_stats = graph_stats.groupby(['gene_type']).mean()[f'{deg_type}_deg']
    rep_stats = graph_stats.groupby(['gene_type', 'Replicate']).mean()[f'{deg_type}_deg']
    rand_mean_stats = rand_graph_stats.groupby(['gene_type']).mean()[f'{deg_type}_deg']
    rand_rep_stats = rand_graph_stats.groupby(['gene_type', 'Replicate']).mean()[f'{deg_type}_deg']
    
    width = 0.2
    x_pos = np.array([1, 2])
    delta = np.array([-width, 0, width])

    rects = {}

    for i_gene_type, gene_type in enumerate(gene_types):
        rects[gene_type] = plt.bar(x_pos + delta[i_gene_type],
                                   [mean_stats.loc[gene_type], rand_mean_stats.loc[gene_type]],
                                    width=width, color=gene_type_color[i_gene_type])

        plt.boxplot([rep_stats.loc[gene_type], rand_rep_stats.loc[gene_type]],
                    positions=x_pos + delta[i_gene_type], 
                    #manage_ticks=False, 
                    widths=0.1, medianprops={'color':'black'})

    plt.xticks(ticks=x_pos, labels=['Evolved', 'Random'])
    ax.yaxis.set_major_locator(mpl.ticker.MultipleLocator(1))
    plt.tick_params(axis='both', which='major', labelsize=tick_fontsize)

    
    plt.ylabel(f'{deg_type.capitalize()}-degree', fontsize=legend_fontsize)
    plt.ylim(0, 14.5)
    plt.grid(axis='y', linestyle=':')
    
    patches = ([mpl.patches.Patch(facecolor=color, edgecolor='black', label=label)
                for color, label in zip(gene_type_color, gene_types)])
    plt.legend(handles=patches, title='Gene type', title_fontsize=legend_fontsize, fontsize=legend_fontsize)

    
    plt.savefig(exp_path.joinpath(f'effective_graph_combined_{deg_type}_degree.pdf'), bbox_inches='tight', dpi=dpi)

In [ ]:
plot_degree_by_env_combined(combined_stats, combined_rand_stats, 'in')

In [ ]:
plot_degree_by_env_combined(combined_stats, combined_rand_stats, 'out')